In [111]:
import os
import glob
import pathlib
import shutil
from Bio import SeqIO

In [112]:
def printlist(list):
    for item in list:
        print(item)

In [113]:
os.chdir('/home/mf019/longread_pangenome/expanded_dataset_analysis/')
cwd = os.getcwd()

In [114]:
print(cwd)

/home/mf019/longread_pangenome/expanded_dataset_analysis


In [115]:
shortread_genomes = [ 
    'GCA_000021405.1',
    'GCA_000166635.2',
    'GCA_000166655.2',
    'GCA_000171735.2',
    'GCA_000172295.2',
    'GCA_000172315.2',
    'GCA_000172335.2',
    'GCA_000181555.2',
    'GCA_000181575.2',
    'GCA_000181715.2',
    'GCA_000181855.2'
]

In [116]:
assemblies = pathlib.Path(cwd).joinpath('assemblies')
all_asms = assemblies.joinpath('all')
complete_asms = assemblies.joinpath('complete')
complete_all_anno = assemblies.joinpath('complete_all')

In [117]:
print(all_asms)

/home/mf019/longread_pangenome/expanded_dataset_analysis/assemblies/all


In [118]:
shortread_genomes = [
    'GCF_000008685.2_ASM868v2_genomic',
    'GCF_000021405.1_ASM2140v1_genomic',
    'GCF_000166635.1_ASM16663v1_genomic', # N40
    'GCF_000166655.1_ASM16665v1_genomic', # JD1
    'GCF_000171735.2_ASM17173v2_genomic',
    'GCF_000172295.2_ASM17229v2_genomic',
    'GCF_000172315.2_ASM17231v2_genomic',
    'GCF_000172335.2_ASM17233v2_genomic',
    'GCF_000181555.2_ASM18155v2_genomic',
    'GCF_000181575.2_ASM18157v2_genomic',
    'GCF_000181715.2_ASM18171v2_genomic',
    'GCF_000181855.2_ASM18185v2_genomic',
]

duplicated_genomes = [
    'GCF_002442595.2_ASM244259v2_genomic',
    'GCF_019464575.1_ASM1946457v1_genomic',
]

files = list(all_asms.glob('*/*.fna'))
print(len(files))
for file in files:
    print(file)

132
/home/mf019/longread_pangenome/expanded_dataset_analysis/assemblies/all/GCA_030439625.1_ASM3043962v1_genomic/GCA_030439625.1_ASM3043962v1_genomic.fna
/home/mf019/longread_pangenome/expanded_dataset_analysis/assemblies/all/GCA_032595535.1_ASM3259553v1_genomic/GCA_032595535.1_ASM3259553v1_genomic.fna
/home/mf019/longread_pangenome/expanded_dataset_analysis/assemblies/all/GCF_000172295.2_ASM17229v2_genomic/GCF_000172295.2_ASM17229v2_genomic.fna
/home/mf019/longread_pangenome/expanded_dataset_analysis/assemblies/all/GCF_000181715.2_ASM18171v2_genomic/GCF_000181715.2_ASM18171v2_genomic.fna
/home/mf019/longread_pangenome/expanded_dataset_analysis/assemblies/all/GCF_040790765.1_ASM4079076v1_genomic/GCF_040790765.1_ASM4079076v1_genomic.fna
/home/mf019/longread_pangenome/expanded_dataset_analysis/assemblies/all/URI45H/URI45H.fna
/home/mf019/longread_pangenome/expanded_dataset_analysis/assemblies/all/GCF_002151465.1_ASM215146v1_genomic/GCF_002151465.1_ASM215146v1_genomic.fna
/home/mf019/long

In [125]:
shortread = []
duplicates = []
bad_qc = []
gca_duplicates = []
incomplete = []

dropped = []
kept = []
dropped_count = 0
for file in files:
    #print(file)
    name = str(file.stem).strip()
    # get the file stuff ready for moving directories.
    gff = f'{file.stem}.gff3'
    #print(gff)
    old_path = file.parent
    new_path = complete_all_anno.joinpath(name)
    old_gff = all_asms.joinpath(name, gff)
    new_gff = complete_asms.joinpath(gff)
    # Okay now let's continue with our checking!
    
    #print(f'Name: {name}')
    if name in shortread_genomes:
        print(f'SHORTREAD: {name}')
        shortread.append(name)
        dropped_count += 1
    elif name in duplicated_genomes:
        print(f'DUPLICATE: {name}')
        duplicates.append(name)
        dropped_count += 1
    elif name == 'UCT95H' or name == 'URI45H' or name == 'K89HH': # K89HH has multiple ospC types.
        bad_qc.append(name)
        dropped_count += 1
        print(f'BAD QC: {name}')
    elif 'GCA_' in name:
        gcf_version = name.replace('GCA_', 'GCF_')
        if gcf_version in [str(f.parent.name) for f in files]: # already deleted the fna, no wonder it's not dropping them. It's checking for the *.fna file which was previously deleted.
            print(f"GCA DUPLICATE: {name}")
            gca_duplicates.append(name)
            dropped_count += 1
            #file.unlink()
        else:
            num_recs = len(list(SeqIO.parse(file, 'fasta')))
            if num_recs < 10:
                print(f'INCOMPLETE: {name}')
                dropped_count += 1
                incomplete.append(name)
            else:
                #print("old:",old_path, '\n', "New:", new_path)
                kept.append(name)
                #shutil.copy(old_gff, new_gff)
                #shutil.copytree(old_path, new_path)
                pass
    else:
        num_recs = len(list(SeqIO.parse(file, 'fasta')))
        #print(f'{name}: {num_recs}')
        if num_recs < 10:
            print(f'INCOMPLETE: {name}')
            dropped_count += 1
            incomplete.append(name)
        else:
            #print("old:",old_path, '\n', "New:", new_path)
            kept.append(name)
            shutil.copy(old_gff, new_gff)
            shutil.copytree(old_path, new_path)

print(f'Number of shortread genomes: {len(shortread)}')
print(f'Number of duplicated genomes: {len(duplicates)}')
print(f'Number of genomes dropped for bad QC: {len(bad_qc)}')
print(f'Number of GCA duplicates: {len(gca_duplicates)}')
print(f'Number of incomplete genomes failing to meet threshhold: {len(incomplete)}')
print(f'total dropped: {dropped_count}')
print(f'Kept: {len(kept)}')

dropped = shortread + duplicates + bad_qc + gca_duplicates + incomplete
print(f' count of dropped files:', len(dropped))

############################################################
#dataset_v4
#Number of shortread genomes: 12
#Number of duplicated genomes: 4
#Number of genomes dropped for bad QC: 2
#Number of GCA duplicates: 20
#Number of incomplete genomes failing to meet threshhold: 13
#total dropped: 51
#Kept: 81
############################################################
#dataset_v5
#Number of shortread genomes: 12
#Number of duplicated genomes: 2
#Number of genomes dropped for bad QC: 3
#Number of GCA duplicates: 20
#Number of incomplete genomes failing to meet threshhold: 13
#total dropped: 50
#Kept: 82
# count of dropped files: 50

INCOMPLETE: GCA_030439625.1_ASM3043962v1_genomic
INCOMPLETE: GCA_032595535.1_ASM3259553v1_genomic
SHORTREAD: GCF_000172295.2_ASM17229v2_genomic
SHORTREAD: GCF_000181715.2_ASM18171v2_genomic
BAD QC: URI45H
INCOMPLETE: GCA_030445025.1_ASM3044502v1_genomic
GCA DUPLICATE: GCA_002151465.1_ASM215146v1_genomic
GCA DUPLICATE: GCA_002442595.2_ASM244259v2_genomic
GCA DUPLICATE: GCA_024662155.1_ASM2466215v1_genomic
GCA DUPLICATE: GCA_019464575.1_ASM1946457v1_genomic
SHORTREAD: GCF_000181575.2_ASM18157v2_genomic
GCA DUPLICATE: GCA_040790775.1_ASM4079077v1_genomic
GCA DUPLICATE: GCA_002151505.1_ASM215150v1_genomic
GCA DUPLICATE: GCA_024662175.1_ASM2466217v1_genomic
SHORTREAD: GCF_000166655.1_ASM16665v1_genomic
GCA DUPLICATE: GCA_002151485.1_ASM215148v1_genomic
INCOMPLETE: GCA_030445005.1_ASM3044500v1_genomic
GCA DUPLICATE: GCA_040790805.1_ASM4079080v1_genomic
GCA DUPLICATE: GCA_024662195.1_ASM2466219v1_genomic
GCA DUPLICATE: GCA_040790755.1_ASM4079075v1_genomic
GCA DUPLICATE: GCA_040790795.1_ASM407

In [123]:
dropped_file = 'misc/dropped_genomes_v5.txt'
kept_file = 'misc/kept_genomes_v5.txt'

with open(dropped_file, 'w') as out:
    out.write("\n".join(dropped))
with open(kept_file, 'w') as out:
    out.write("\n".join(kept))

In [81]:
#files = all_asms.glob('*/*.fna')
#for genome in shortread_genomes:
#    print(os.listdir(all_asms.joinpath(genome)))

In [84]:
dropped_file_v3 = 'misc/dropped_genomes_v3.txt'
kept_file_v3 = 'misc/kept_genomes_v3.txt'

kept_v4 = []
dropped_v4 = []
with open(kept_file, 'r') as infi:
    lines = infi.readlines()
    for line in lines:
        kept_v4.append(line.strip())

with open(dropped_file, 'r') as infi:
    lines = infi.readlines()
    for line in lines:
        dropped_v4.append(line.strip())


kept_v3 = []
dropped_v3 = []
with open(kept_file_v3, 'r') as infi:
    lines = infi.readlines()
    for line in lines:
        kept_v3.append(line.strip())

with open(dropped_file_v3, 'r') as infi:
    lines = infi.readlines()
    for line in lines:
        dropped_v3.append(line.strip())

print("kept:")
print("v3:", len(kept_f_v3))
print("v4:", len(kept_f_v4))
print(set(kept_f_v3)-set(kept_f_v4)) # see below:

# Newly Dropped:
#GCF_000166635.1_ASM16663v1_genomic # N40: SR
#GCF_000166655.1_ASM16665v1_genomic # JD1: SR
#GCF_002442595.2_ASM244259v2_genomic # B331: LR dupe  with UNY167
#GCF_019464575.1_ASM1946457v1_genomic # B500: LR dupe with UNY196

#GCF_024662155.1_ASM2466215v1_genomic # B408: LR dupe with UNY183
#GCF_024662175.1_ASM2466217v1_genomic # B379: LR dupe with UNY177


kept:
v3: 87
v4: 81
{'GCF_024662155.1_ASM2466215v1_genomic', 'GCF_019464575.1_ASM1946457v1_genomic', 'GCF_002442595.2_ASM244259v2_genomic', 'GCF_000166655.1_ASM16665v1_genomic', 'GCF_024662175.1_ASM2466217v1_genomic', 'GCF_000166635.1_ASM16663v1_genomic'}


In [109]:
curr_dirs = list(complete_all_anno.glob('*'))
for file in curr_dirs:
    name = str(file).split('/')[-1]
    #print(name)
    if name in dropped_v4: # is only 6 genomes.
        #print(name)
        print(file)
        shutil.rmtree(file)

GCF_040790765.1_ASM4079076v1_genomic
GCF_002151465.1_ASM215146v1_genomic
URI87H
URI34H
URI88H
URI33H
UCT110H
URI39H
URI91H
UCT35H
UWI247P
URI120H
URI107H
GCF_002151505.1_ASM215150v1_genomic
UWI263P
GCF_040790795.1_ASM4079079v1_genomic
GCF_040790785.1_ASM4079078v1_genomic
URI89H
URI42H
URI44H
UCT109H
URI40H
URI117H
URI47H
B500PP
URI86H
GCF_040790715.1_ASM4079071v1_genomic
ESI36H
GCF_040790755.1_ASM4079075v1_genomic
URI36H
UNY208P
GCF_040790745.1_ASM4079074v1_genomic
ESI26H
UCT31H
GCF_040790805.1_ASM4079080v1_genomic
ESI403H
URI56H
XYZ459H
UCT30H
GCF_040790735.1_ASM4079073v1_genomic
GCF_040819585.1_PFhe_I_PB_Ill_cons_genomic
URI103H
UCT29H
UNY1128P
URI112H
UNY1032P
UWI248P
UNY203P
UCT96H
GCF_019134655.1_ASM1913465v1_genomic
ESI425H
UCT32H
UNY990P
UNY193P
UCT113H
URI93H
UNY169P
UWI283P
UNY1090P
UNY1083P
URI102H
B418PP
URI41H
B331PP
UNY172P
GCF_040790775.1_ASM4079077v1_genomic
UNY149P
GCF_024662195.1_ASM2466219v1_genomic
UCT92H
URI118H
UCT50H
URI101H
UNY1085P
URI46H
URI48H
GCF_002151485.1_

In [110]:
# let's check to see if we have everything sorted finally.
print(len(list(complete_all_anno.glob('*'))))

81
